In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 创建同域数据集

指定一个源数据集名字，比如 IMDB，然后从中随机选取若干条文本，分成两半，形成该数据集的两个同域数据集

In [11]:
from input_corpus import TextCorpusDispatcher
from subject_model import ModelSubject_GPT2
from tqdm.notebook import tqdm
from path import *
import torch

# 源数据集名字，任务列表
dataset_names = [
    'ag_news',
    'stanfordnlp/imdb',
]

# 超参数
# - 选择GPT2模型
# - 每个源数据集要分成两个同域小数据集，每个小数据集包含5000个样本
# - 每个小数据集的batch size为10，共500个batch
model = ModelSubject_GPT2('gpt2')
num_each = 5000
batch_size = 10

# 对每个源数据集造两个同域小数据集
for dataset_name in dataset_names:
    print(f'Processing {dataset_name}...')

    # 两个同域小数据集的 batches
    batches_1, batches_2 = TextCorpusDispatcher(
        corpus_name=dataset_name,
        num_each=num_each,
        batch_size=batch_size,
    )

    index = 1
    for batches in [batches_1, batches_2]:
        # 收集每个同域小数据集上的激活次数
        ret = torch.zeros(model.n_layer, model.n_neuron, device=model.device)
        for batch in tqdm(batches.batches, desc=f'Processing batches {index}', total=len(batches.batches)):
            ret += model.inference(batch.texts)
            
        dataset_name = dataset_name.split('/')[-1]
        # 保存激活次数为文件
        torch.save(
            ret, 
            times(f'gpt2_{dataset_name}_{num_each}_{index}'),
        )
        
        index += 1

/home/v-ruihuang/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Processing ag_news...


Processing batches 1:   0%|          | 0/500 [00:00<?, ?it/s]

Processing batches 2:   0%|          | 0/500 [00:00<?, ?it/s]

Processing stanfordnlp/imdb...


Processing batches 1:   0%|          | 0/500 [00:00<?, ?it/s]

Processing batches 2:   0%|          | 0/500 [00:00<?, ?it/s]